In [1]:
from scipy.sparse import csr_matrix, coo_matrix
from symmer.chem import MoleculeBuilder
from symmer.symplectic import random_PauliwordOp, symplectic_to_sparse_matrix
from symmer.projection import QubitTapering, CS_VQE, StabilizerIdentification
from symmer.utils import exact_gs_energy
import multiprocessing as mp
import numpy as np
import time
import json

with open('../data/molecule_data.json', 'r') as jfile:
    molecule_geometries = json.load(jfile)

In [2]:
speciesname = 'BeH2_STO-3G_SINGLET'
# build the molecule
mol_data = molecule_geometries[speciesname]
atoms  = mol_data['atoms']
coords = mol_data['coords']
geometry = list(zip(atoms, coords))
molecule = MoleculeBuilder(geometry=geometry, charge=0, basis='STO-3G', spin=0, run_fci=True, print_info=True)

# taper the Hamiltonian
taper_hamiltonian = QubitTapering(molecule.H_q)
hf_array = molecule.H_fermion.hf_comp_basis_state
taper_hamiltonian.stabilizers.update_sector(hf_array)
ham_tap = taper_hamiltonian.taper_it(ref_state=hf_array)
ucc_tap = taper_hamiltonian.taper_it(aux_operator=molecule.T_q, ref_state=hf_array)
hf_tapered = taper_hamiltonian.tapered_ref_state

# initiate stabilizer identification classes
CC_stabilizers = StabilizerIdentification(ucc_tap)
cs_vqe = CS_VQE(ham_tap, hf_tapered, basis_weighting_operator=ucc_tap, noncontextual_form='diag')

Molecule geometry:
Be	0.0	0.0	0.0
H	0.0	0.0	1.29055
H	0.0	0.0	-1.29055

HF converged?   True
CCSD converged? True
FCI converged?  True

HF energy:   -15.561352807666323
MP2 energy:  -15.583725038009472
CCSD energy: -15.594378365854427
FCI energy:  -15.594746145660254


Number of qubits: 14


In [5]:
S = CC_stabilizers.symmetry_basis_by_subspace_dimension(6)
H_cs = cs_vqe.project_onto_subspace(S)
CC_cs = cs_vqe.project_onto_subspace(S, aux_operator=ucc_tap)
exact_gs_energy(H_cs.to_sparse_matrix)[0] - molecule.fci_energy

0.000852347818089072

In [6]:
CC_cs

0.003-0.000j IIIIIY +
0.003+0.000j IIIZZY +
0.003-0.000j IZZIIY +
0.003+0.000j IZZZZY +
-0.003+0.000j ZIIIZY +
-0.003+0.000j ZIIZIY +
-0.003+0.000j ZZZIZY +
-0.003+0.000j ZZZZIY +
0.000-0.000j IIIIYI +
0.000+0.000j IIIZYZ +
0.000-0.000j IZZIYI +
0.000+0.000j IZZZYZ +
-0.000+0.000j ZIIIYZ +
-0.000+0.000j ZIIZYI +
-0.000+0.000j ZZZIYZ +
-0.000+0.000j ZZZZYI +
0.003+0.000j IIIIXY +
0.003+0.000j IIIIYX +
-0.003+0.000j ZZZZXY +
-0.003+0.000j ZZZZYX +
-0.007+0.000j IIIYII +
-0.007+0.000j IIIYZZ +
0.007+0.000j ZZZYIZ +
0.007+0.000j ZZZYZI +
0.003-0.000j IIIXYI +
0.003-0.000j IIIYXI +
0.003+0.000j IZZXYI +
0.003+0.000j IZZYXI +
-0.003+0.000j ZIIXYZ +
-0.003+0.000j ZIIYXZ +
-0.003+0.000j ZZZXYZ +
-0.003+0.000j ZZZYXZ +
-0.007+0.000j IIYIII +
-0.007+0.000j IIYIZZ +
0.007-0.000j ZZYZIZ +
0.007-0.000j ZZYZZI +
-0.007+0.000j IYIIII +
-0.007+0.000j IYIIZZ +
0.007-0.000j ZYZZIZ +
0.007-0.000j ZYZZZI +
0.003-0.000j XIIIIY +
0.003+0.000j XIIZZY +
0.003-0.000j XZZIIY +
0.003+0.000j XZZZZY +
-0.003+0.000

In [7]:
from symmer.symplectic import AnsatzOp, ObservableOp

In [8]:
CC_truncated = CC_cs.sort()

obs = ObservableOp(H_cs.symp_matrix, H_cs.coeff_vec)
anz = AnsatzOp(CC_truncated.symp_matrix, CC_truncated.coeff_vec)
ref_state = hf_tapered[cs_vqe.free_qubit_indices]

In [9]:
obs.evaluation_method = 'statevector' #'trotter_rotations'

In [10]:
obs.VQE(ansatz_op=anz, ref_state=ref_state)[0]['fun'] - molecule.fci_energy

0.0008537635671235222

In [69]:
from multiprocessing.pool import ThreadPool

In [82]:
help(ThreadPool)

Help on class ThreadPool in module multiprocessing.pool:

class ThreadPool(Pool)
 |  ThreadPool(processes=None, initializer=None, initargs=())
 |  
 |  Class which supports an async version of applying functions to arguments.
 |  
 |  Method resolution order:
 |      ThreadPool
 |      Pool
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, processes=None, initializer=None, initargs=())
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  Process(ctx, *args, **kwds)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from Pool:
 |  
 |  __del__(self, _warn=<built-in function warn>, RUN='RUN')
 |      # Copy globals as function locals to make sure that they are available
 |      # during Python shutdown when the Pool is destroyed.
 |  
 |  __enter__(self)
 |  
 |  __exit__

In [ ]:
obs.evaluation_method = 'sampled'
obs.n_shots=1

def gradient(
    ansatz_op: AnsatzOp,
    ref_state: np.array
    ) -> np.array:
    """ multiprocessing allows partial gradients to be computed simultaneously
    """
    pool = ThreadPool(mp.cpu_count())
    return pool.starmap(
        obs.parameter_shift_at_index, 
        [(i, ansatz_op, ref_state) for i in range(ansatz_op.n_terms)]
    )

gradient(anz, ref_state)

IOStream.flush timed out
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



In [94]:
obs.evaluation_method = 'statevector'

def gradient(
    ansatz_op: AnsatzOp,
    ref_state: np.array
    ) -> np.array:
    """ multiprocessing allows partial gradients to be computed simultaneously
    """
    pool = mp.Pool(mp.cpu_count())
    return pool.starmap(
        obs.parameter_shift_at_index, 
        [(i, ansatz_op, ref_state) for i in range(ansatz_op.n_terms)]
    )

%timeit gradient(anz, ref_state)

225 ms ± 8.44 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
def convert_to_sparse(self):
    out_matrix = csr_matrix( ([],([],[])),
                              shape=(2**self.n_qubits,2**self.n_qubits)
                              )
    for Pvec_single, coeff_single in zip(self.symp_matrix, self.coeff_vec):
        out_matrix += symplectic_to_sparse_matrix(Pvec_single, coeff_single)
    return out_matrix

In [4]:
from functools import reduce

def convert_to_sparse_parallel(self):
    # initiate a pool for parallel processing of Pauli multiplication
    pool = mp.Pool(mp.cpu_count(), maxtasksperchild=10)
    matrix_elements = []
    # loop over the Pauli terms of right-hand PauliwordOp and parallelize
    for Pvec_single, coeff_single in zip(self.symp_matrix, self.coeff_vec):
        pool.apply_async(
            symplectic_to_sparse_matrix, 
            args=(Pvec_single, coeff_single), 
            callback=lambda x:matrix_elements.append(x)
        )
    # close the pool and let all the processes complete    
    pool.close()
    pool.join()  # postpones the execution of next line of code until all processes in the queue are done.
    print('reduce phase')
    return reduce(lambda x,y:x+y, matrix_elements)

In [5]:
start = time.time()
sparse_para = convert_to_sparse_parallel(P)
end = time.time()

print(end-start)

Process ForkPoolWorker-495:
Process ForkPoolWorker-492:
Process ForkPoolWorker-348:
Process ForkPoolWorker-496:
Process ForkPoolWorker-346:
Process ForkPoolWorker-347:
Process ForkPoolWorker-345:
Process ForkPoolWorker-462:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/tweaving/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/tweaving/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/tweaving/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/tweaving/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/tweaving/anaconda3/lib

Traceback (most recent call last):
  File "/home/tweaving/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_59361/689471847.py", line 2, in <module>
    sparse_para = convert_to_sparse_parallel(P)
  File "/tmp/ipykernel_59361/2297358586.py", line 16, in convert_to_sparse_parallel
    pool.join()  # postpones the execution of next line of code until all processes in the queue are done.
  File "/home/tweaving/anaconda3/lib/python3.8/multiprocessing/pool.py", line 662, in join
    self._worker_handler.join()
  File "/home/tweaving/anaconda3/lib/python3.8/threading.py", line 1011, in join
    self._wait_for_tstate_lock()
  File "/home/tweaving/anaconda3/lib/python3.8/threading.py", line 1027, in _wait_for_tstate_lock
    elif lock.acquire(block, timeout):
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call la

TypeError: object of type 'NoneType' has no len()

Process ForkPoolWorker-503:
Process ForkPoolWorker-502:
Process ForkPoolWorker-498:
Process ForkPoolWorker-499:
Process ForkPoolWorker-504:
Process ForkPoolWorker-497:
Process ForkPoolWorker-501:
Process ForkPoolWorker-500:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/tweaving/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/tweaving/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/tweaving/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/tweaving/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/tweaving/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._tar

In [ ]:
start = time.time()
sparse_strd = convert_to_sparse(P)
end = time.time()

print(end-start)

In [ ]:
start = time.time()
sparse_qiskit = P.to_PauliSumOp.to_spmatrix()
end = time.time()

print(end-start)

In [ ]:
from openfermion import get_sparse_operator

start = time.time()
sparse_open = get_sparse_operator(P.to_QubitOperator)
end = time.time()

print(end-start)

In [ ]:
exact_gs_energy(sparse_old)[0]

In [ ]:
exact_gs_energy(sparse_para)[0]

In [ ]:
exact_gs_energy(sparse_strd)[0]

In [ ]:
exact_gs_energy(sparse_qiskit)[0]